# Ingredients of size spectrum models

This document introduces the ingredients of the community size spectrum model (CSSM)

Much of this can be found in [the vignette of the R package "mizer"](https://mran.microsoft.com/snapshot/2017-02-04/web/packages/mizer/vignettes/mizer_vignette.pdf)

Other resources:
* [Benoît & Rochet 2004](https://doi.org/10.1016/S0022-5193(03)00290-X)
* [Blanchard et al, 2009](https://doi.org/10.1111/j.1365-2656.2008.01466.x)
* [Hartvig et al, 2011](https://doi.org/10.1016/j.jtbi.2010.12.006)
* [Blanchard et al, 2012](https://doi.org/10.1098/rstb.2012.0231)
* [Scott et al, 2014](https://doi.org/10.1111/2041-210X.12256)
* [Blanchard et al, 2014](https://doi.org/10.1111/1365-2664.12238)

For notation we follow [Blanchard et al, 2014](https://doi.org/10.1111/1365-2664.12238) and [Scott et al, 2014](https://doi.org/10.1111/2041-210X.12256).

In [ ]:
import numpy
import scipy.stats
%matplotlib widget
from matplotlib import pyplot

In [ ]:
# Set up a grid for wet mass - to be used for plotting
w_min, w_max = 0.0001, 1e6
w_grid = 10.**numpy.linspace(numpy.log10(w_min), numpy.log10(w_max), 1000)

# It all begins with an individual fish

## Size-based prey preference

The preference of a predator with wet mass $w$ for prey of $w_p$ takes the shape of a lognormal distribution:

$$\phi(w_p/w) = \exp\left[\frac{-(\ln{(\beta w_p / w)})^2}{2\sigma^2}\right]$$

The optimal predator : prey mass ratio $\beta$ is 100 in all studies listed above, but the width of the distribution varies:
* for a single species: $\sigma = 1$
* for a community: $\sigma = 2$ ([Scott et al, 2014](https://doi.org/10.1111/2041-210X.12256)) or $\sigma = \ln{10} \approx 2.3$ ([Blanchard et al, 2012](https://doi.org/10.1098/rstb.2012.0231))

In [ ]:
beta = 100.                        # preferred predator : prey amss ratio
sigmas = [1, 2., numpy.log(10.)]   # standard deviation of selectivity function (ln g)
w_predator = 100.

fig, ax = pyplot.subplots()
for sigma in sigmas:
    phi = numpy.exp(-(numpy.log(w_grid) - numpy.log(w_predator / beta))**2 / (2. * sigma**2))
    ax.semilogx(w_grid, phi, label='$\sigma = %.2f$' % sigma)

ax.axvline(w_predator, linestyle='--', color='k', label='predator mass')
ax.set_ylabel('predator preference (-)')
ax.set_xlabel('prey mass (g)')
ax.grid()
ax.legend();

## Predator-prey encounter and clearance rate

Available food (g/m³) is essentially the sum of all available prey, weighted by preference. With prey expressed as biomass density (concentration per unit individual mass), that sum becomes an integral over individual prey mass:

$$E_a = \int{\phi(w_p/w)} c(w_p) dw_p$$ with $c(w_p)$ the concentration density (1/m³) of prey with wet mass $w_p$

Encountered food (g/individual predator/yr) is the product rate of available food and the clearance rate $V$ (m³/yr):

$$E_e = V E_a$$

This rate describes the volume of water searched for prey per unit time. It is an allometric function of wet mass: $V=\gamma w^q$.

In [ ]:
# A few different allometric relationships for clearance rate
benoit_blanchard = 640 * w_grid**0.82
acuna = 10.**14.076 * w_grid**0.926 * 1e-3 * 365 * numpy.exp(-0.65 / 8.62e-5 / (273.15 + 13))  # convert from L to m3, from 1/d to 1/yr, apply Arrhenius temperature correction
kiorboe = 10.**7.31 * (0.1 * w_grid)**(1.01) * 1e-6 * 365 * 2.8**-0.2  # convert from ml to m3, from 1/d to 1/yr, apply Q10 temperature correction

fig, ax = pyplot.subplots()

ax.loglog(w_grid, benoit_blanchard, '-', label='Benoît et al. 2004, Blanchard et al. 2009')
ax.loglog(w_grid[w_grid<1e4], acuna[w_grid<1e4], '-', label='Acuña et al. 2009')
ax.loglog(w_grid[w_grid<10.], kiorboe[w_grid<10.], '-', label='Kiørboe et al. 2011')
ax.set_xlabel('individual mass (g)')
ax.set_ylabel('clearance rate (m³/yr)')
ax.axhline(365., linestyle='--', color='k', label='1 m³/d')
ax.legend()
ax.grid()

# Load and plot the Acuña dataset
import csv
ww, clearance = [], []
with open('../datasets/Acuna_et_al_2012_table_S1.dat') as f:
    for name2value in csv.DictReader(f, delimiter='\t'):
        if name2value['functional type'] in ('SF',):
            temp_corr = numpy.exp(-0.65 / 8.62e-5 * (1./ (273.15 + 13.) - 1./ (273.15 + float(name2value['Temp']))))
            ww.append(float(name2value['WW']))
            clearance.append(float(name2value['Clearance']) * temp_corr * 1e-3 * 365)
ax.loglog(ww, clearance, '.', mec='gray', mfc='w', label='Acuña et al. 2009 Fig2A', zorder=-1);

## Ingestion

By default, ingestion is described with Type II ("Michaelis-Menten") functional response. Its maximum rate (g/yr) is yet another allometric function of wet mass: $h w^n$. The value of the functional response (0-1), or "food level", equals:

$$f = \frac{E_e}{E_e + h w^n}$$

This food level is thought to lie around 0.6 in natural systems (Hartvig et al. 2011).

Actual ingestion is the product of th maximum ingestion rate and the food level:

$$I = h w^n f$$

For models without saturation (Blanchard et al. 2009), all encountered food is eaten.
In our implementation, this is achieved by setting $h \rightarrow \infty$, which means $I \rightarrow E_e$.
In practice we substitute a "large number" (1e9) for $\infty$.

In [ ]:
# Scott et al. 2014 parameterisation for community model
h = 10.    # maximum ingestion rate (g/yr) for individual of 1 g
n = 2./3.  # allometric scaling of maximum ingestion rate

# Disable saturation altogether to replicate Blanchard et al. 2009 (I -> E_e)
h = 1e9    # maximum ingestion rate (g/yr) for individual of 1 g

fig, ax = pyplot.subplots()
E_a = numpy.linspace(0, 20., 1000)
for w in (0.001, 1., 100., 10000):
    V = 10.**14.076 * w**0.926 * 1e-3 * 365 * numpy.exp(-0.65 / 8.62e-5 / (273.15 + 13))
    E_e = V * E_a
    I_max = h * w**n
    f = E_e / (E_e + I_max)
    I = I_max / w * f
    ax.plot(E_a, I, '-', label='predator of %s g' % w)
ax.legend()
ax.grid()
ax.set_xlabel('available prey (g/m³)')
ax.set_ylabel('specific ingestion (g prey/g predator/yr)');

## Growth

In Blanchard et al. (2009) growth is a fixed fraction of ingested biomass.
That fraction is the Gross Growth Conversion efficiency, $\alpha$. As it encapsulates all metabolic losses including respiration,
its value is low (0.2)

NB In species-specific (and trait-based) size spectrum models, there is an additional "standard metabolism" term that is independent of ingestion.
In that case, $\alpha$ is the assimilation efficiency that takes higher values (0.6).

## Fisheries mortality and its size selectivity
* linear: [Blanchard et al, 2009](https://doi.org/10.1111/j.1365-2656.2008.01466.x)
* knife-edge: [Blanchard et al, 2012](https://doi.org/10.1098/rstb.2012.0231)
* sigmoid: [Blanchard et al, 2014](https://doi.org/10.1111/1365-2664.12238)

In [ ]:
fig, (ax1, ax2, ax3) = pyplot.subplots(ncols=3, figsize=(12,5))

# Knife-edge
f1 = numpy.full_like(w_grid, 0.8); f1[w_grid < 1.25] = 0
f2 = numpy.full_like(w_grid, 0.2); f2[w_grid < 1.25] = 0
ax1.semilogx(w_grid, f1,'-')
ax1.semilogx(w_grid, f2,'--')
ax1.set_title('knife-edge (fishing_type=1)')

# Sigmoid
w25 = numpy.log(10.)   # wet mass where fishing pressure is 25% of maximum
w50 = numpy.log(100.)  # wet mass where fishing pressure is 50% of maximum
S2 = numpy.log(3)/(w50-w25)
S1 = S2 * w50
f = 0.8 * 1. / (1. + numpy.exp(S1 - S2 * numpy.log(w_grid)))
ax2.semilogx(w_grid, f, '-')
ax2.set_title('sigmoid (fishing_type=2)')

# Linear
f = 0.09 * numpy.log10(w_grid) + 0.04; f[w_grid < 10.] = 0
ax3.semilogx(w_grid, f,'-')
ax3.set_title('linear (fishing_type=3)')

ax1.set_ylabel('fisheries mortality (1/yr)')
for ax in (ax1, ax2, ax3):
    ax.set_xlabel('individual wet mass (g)')
    ax.set_xlim(.001, 1e6)
    ax.set_ylim(None, 1)
    ax.grid()

## "Other" mortality

### Intrinsic mortality

"Mortality from sources other than predation and starvation is assumed to be constant within a species
and inversely proportional to generation time" (mizer vignette)

$$\mu_b = \mu_0 w^{z_0}$$

* $\mu_0 = 0.2, z_0 = -0.25$ (Blanchard et al. 2009, 2012)
* $\mu_0 = 0.1, z_0 = n-1=-1/3$ (Scott et al. 2012)

### Senescent mortality (Blanchard et al. 2009, 2012 only)

"Although we have no data to support such a functional form, it seems intuitively obvious that it must occur at the limit in unexploited fish populations to prevent the buildup of very large fish" ([Hall et al, 2006](https://doi.org/10.1139/f06-039))

$$\mu_s = 0.2 (w/w_s)^{z_s}$$ with $z_s=0.3$ and $w_s$ = 1 kg

In [ ]:
fig, ax = pyplot.subplots()

mu_0 = 0.2
z0exp = -0.25
mu_b = mu_0 * w_grid**z0exp

z_spre = 0.2 # Blanchard et al. 2009, 2012
w_s = 1000.
z_s = 0.3
mu_s = z_spre * (w_grid / w_s)**z_s

ax.semilogx(w_grid, mu_b, label='intrinsic mortality $\mu_b$')
ax.semilogx(w_grid, mu_s, label='senescent mortality $\mu_s$')
ax.semilogx(w_grid, mu_b + mu_s, '--k', label='combined')
ax.set_xlabel('wet mass (g)')
ax.set_ylabel('mortality (1/yr)')
ax.grid()
ax.legend();

## Temperature dependence

[Blanchard et al. (2012)](https://doi.org/10.1098/rstb.2012.0231) apply an Arrhenius relationship to all rates:

$$\exp \left[ c1 - \frac{E_a}{k (T + 273.15)} \right]$$

* $E_a$ = 0.63 eV is the activation energy
* $k$ the Boltzmann constant ($8.62 \times 10^{-5}$ eV/K)
* $T$ the temperature in °C

$c1=25.55$ represents $E_a / (k (T + 273.15)$ for a reference temperature, which can be reconstructed to have been 13°C.

In [ ]:
# Illustrate temperature dependence introduced by Blanchard et al. (2012, 10.1098/rstb.2012.0231)
# For comparison add curve for Q10=2
E_a = 0.63   # activation energy (eV)
T_ref = 13.  # reference temperature
k = 8.62e-5  # Boltzmann constant (eV K-1)
c1 = E_a / k / (T_ref + 273.) # minus natural log of scale factor at reference temperature (equal to c1 in Blanchard et al.)
print('c1 = %.2f (cf. Blanchard et al. 2012)' % c1)
T = numpy.linspace(-2., 30, 1000)
tau = numpy.exp(c1 - E_a / k / (T + 273.))
fig, ax = pyplot.subplots()
ax.plot(T, tau, '-', label='Arrhenius with $E_a = 0.63$ (Blanchard et al. 2012)')
ax.plot(T, 2.8**((T - T_ref) / 10.), '--', label='$Q_{10}=2.8$ (Kiørboe 2011, Hansen et al. 1997)')
ax.plot(T, 2.**((T - T_ref) / 10.), '--', label='$Q_{10}=2$')
ax.set_xlabel('temperature (degrees Celsius)')
ax.set_ylabel('relative rate (-)')
ax.axvline(T_ref, linestyle=':', color='k')
ax.legend()
ax.grid()

# From individual to population

So far we have talked about the behaviour of an individual predator.
The behaviour of this individual is completely defined by its wet mass (that is is only "state").
A population of such individuals is described by a structured population model.
Mathematically this is a partial differential equation
for predator biomass density $W$ (g/m²) per unit individual wet mass $w$ (g):

$$\frac{\partial W}{\partial t} = \frac{\partial}{\partial w}\left[g(w) W\right] - \mu W$$

Here, $g(w)$ is the growth rate (g/time) of an individual of wet mass $w$.
$\mu$ is the combined mortality (1/time) due to predation, fisheries, intrinsic and senescence. Note: some authors formulate the model in terms of predator _abundancy_ density $N = W / w$.

This is solved numerically with a first-order upwind discretisation.
This is the moment the continuous spectrum becomes represented by a number of bins.